# Continuous Interation

The idea behind continuous integration is to automate away the testing of your code.

We will be using it for our projects.

The basic workflow goes something like this:

1. You work on your part of the code in your own branch or fork
2. On every commit you make and push to github, your code is automatically tested on a fresh machine on Travis CI. This ensures that there are no specific dependencies on the structure of your machine that your code needs to run, and also ensures that your changes are sane
3. Now you submit a pull request to `master` in the "sacred" repo. The repo manager creates a branch off `master`. 
4. This branch is also set to run tests on Travis. If all tests pass (remember other code may have been added to master in the time you were working on your own branch), then the pull request is accepted and your code becomes part of master.

We use github to integrate our math library with Travis CI and Coveralls. Note that this is not the only workflow people use. Google git..github..workflow and feel free to choose another one for your group. Document your workflow on your project's README.

In [1]:
%%file amath.py

def myaverage(l:list)->float:
    """
    Calculate the average of list l
    
    Examples:
    
    >>> myaverage([1,2])
    1.5
    
    """
    n = len(l)
    if n==0:
        raise ValueError("cant calculate mean of length 0 list")
    try:
        thesum = sum(l)
    except:
        raise TypeError("Cannot sum things of different types")
    average = thesum/n
    return average

def mymedian(l:list)->float:
    """
    Calculate the average of list l
    
    Examples:
    
    >>> mymedian([1,2,3])
    2
    
    >>> mymedian([1,2,3,4])
    2.5
    """
    try:
        lsorted = sorted(l)
    except:
        raise TypeError("Unable to sort array")
    n = len(lsorted)
    if n==0:
        raise ValueError("cant calculate median of length 0 list")
    mididx = len(lsorted)//2
    if len(lsorted) % 2 == 0: #even
        return (lsorted[mididx-1] + lsorted[mididx])/2
    else:
        return lsorted[mididx]

Writing amath.py


In [2]:
%%file test_amath.py

from pytest import raises
from amath import myaverage, mymedian

def test_mymath_mean():
    assert myaverage([9,3]) == 6

def test_char():
    with raises(TypeError):
        myaverage(['a',3])

def test_mymath():
    assert mymedian([9,3, 6]) == 6
    
def test_zero_median():
    with raises(ValueError):
        mymedian([])
        
def test_char_median():
    with raises(TypeError):
        mymedian(['a', 3])

Writing test_amath.py


In [4]:
!py.test --doctest-modules --cov --cov-report term-missing amath.py test_amath.py

============================= test session starts ==============================
platform darwin -- Python 3.5.1, pytest-2.8.1, py-1.4.30, pluggy-0.3.1
rootdir: /Users/rahul/Projects/private/cs207/lecswithlabs/week5, inifile: 
plugins: cov-2.2.1
collected 7 items 

amath.py ..
test_amath.py .....
--------------- coverage: platform darwin, python 3.5.1-final-0 ----------------
Name            Stmts   Miss  Cover   Missing
---------------------------------------------
amath.py           22      1    95%   14
test_amath.py      15      0   100%   
---------------------------------------------
TOTAL              37      1    97%   

=========================== 7 passed in 0.04 seconds ===========================


## Setup

Create a repo for yourself called `cs207test`. Below, I've used my github id, replce with yours. Then clone it down to your machine

In [3]:
!rm -rf /tmp/cs207test
!git clone git@github.com:rahuldave/cs207test.git /tmp/cs207test
    

Cloning into '/tmp/cs207test'...
remote: Counting objects: 50, done.
remote: Total 50 (delta 0), reused 0 (delta 0), pack-reused 50
Receiving objects: 100% (50/50), 5.52 KiB | 0 bytes/s, done.
Resolving deltas: 100% (19/19), done.
Checking connectivity... done.


Copy our library file and test file in there.

In [6]:
!cp amath.py test_amath.py /tmp/cs207test/

In [7]:
%%file /tmp/cs207test/setup.cfg
[pytest]
addopts = --doctest-modules --cov-report term-missing --cov amath

Overwriting /tmp/cs207test/setup.cfg


### Travis CI

![travis process](travis-process.png)

Create an account on Travis CI and set this repo up for continuous integration once this repo can be seen on Travis.

We then creare an instructio to Travis to make sure that

1. python is installed
2. its python 3.5
3. pytest is installed

In [10]:
%%file /tmp/cs207test/.travis.yml
language: python
python:
    - "3.5"
before_install:
    - pip install pytest pytest-cov
script:
    - py.test

Overwriting /tmp/cs207test/.travis.yml


In [11]:
%%bash
pushd /tmp/cs207test
git add .
git commit -m "travis integration" -a
git push
popd

/tmp/cs207test ~/Projects/private/cs207/FALL/labs
On branch master
Your branch is up-to-date with 'origin/master'.
nothing to commit, working directory clean
~/Projects/private/cs207/FALL/labs


Everything up-to-date


![build passing](travis-build-passing.png)

At this point you be able to see your build on travis and if and how your tests pass.

### Coveralls Integration

![coveralls on](coveralls-on.png)

Next, create an account on "coveralls", connect your github, and turn coveralls integration on.

Then we have to give Travis some new instructions:

In [12]:
%%file /tmp/cs207test/.travis.yml
language: python
python:
    - "3.5"
before_install:
    - pip install pytest pytest-cov
    - pip install coveralls
script:
    - py.test
after_success:
    - coveralls

Overwriting /tmp/cs207test/.travis.yml


In [13]:
%%bash
pushd /tmp/cs207test
git add .
git commit -m "added coveralls" -a
git push
popd

/tmp/cs207test ~/Projects/private/cs207/FALL/labs
[master 3dbeb2c] added coveralls
 1 file changed, 4 insertions(+), 1 deletion(-)
~/Projects/private/cs207/FALL/labs


To git@github.com:rahuldave/cs207test.git
   a5f0d97..3dbeb2c  master -> master


![coveralls on](coveralls-repo.png)

![coveralls on](coveralls-lines.png)

We want our github repo to reflect our build status on Travis CI and our coverage status. So we'll add this to our README as badges

In [14]:
%%file /tmp/cs207test/README.md

# cs207test

[![Build Status](https://travis-ci.org/rahuldave/cs207test.svg?branch=master)](https://travis-ci.org/rahuldave/cs207test)

[![Coverage Status](https://coveralls.io/repos/github/rahuldave/cs207test/badge.svg?branch=master)](https://coveralls.io/github/rahuldave/cs207test?branch=master)

Overwriting /tmp/cs207test/README.md


In [15]:
%%bash
pushd /tmp/cs207test
git add .
git commit -m "added badges" -a
git push
popd

/tmp/cs207test ~/Projects/private/cs207/FALL/labs
On branch master
Your branch is up-to-date with 'origin/master'.
nothing to commit, working directory clean
~/Projects/private/cs207/FALL/labs


Everything up-to-date


![badges](github-badges.png)